In [1]:
from torchmetrics import JaccardIndex
import torch

In [2]:
target = torch.randint(0, 2, (10, 25, 25))
pred = torch.tensor(target)

/tmp/ipykernel_1135339/281148576.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = torch.tensor(target)


In [3]:
target

tensor([[[1, 1, 1,  ..., 1, 0, 0],
         [1, 0, 1,  ..., 1, 0, 1],
         [1, 0, 1,  ..., 1, 1, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         [1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 1, 0, 0],
         ...,
         [1, 1, 0,  ..., 0, 1, 0],
         [1, 1, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 1, 0, 0]],

        [[0, 1, 0,  ..., 1, 0, 0],
         [1, 1, 1,  ..., 1, 0, 0],
         [0, 0, 1,  ..., 1, 1, 0],
         ...,
         [0, 1, 0,  ..., 0, 0, 1],
         [0, 1, 0,  ..., 0, 0, 0],
         [1, 0, 1,  ..., 1, 0, 1]],

        ...,

        [[1, 1, 0,  ..., 0, 1, 0],
         [1, 0, 1,  ..., 1, 1, 0],
         [0, 0, 0,  ..., 1, 0, 0],
         ...,
         [1, 0, 1,  ..., 0, 1, 0],
         [0, 0, 1,  ..., 0, 1, 0],
         [1, 0, 0,  ..., 0, 0, 1]],

        [[0, 0, 1,  ..., 1, 1, 0],
         [0, 0, 1,  ..., 0, 1, 0],
         [0,

In [4]:
target.shape

torch.Size([10, 25, 25])

In [5]:
pred[2:5, 7:13, 9:15] = 1 - pred[2:5, 7:13, 9:15]
jaccard = JaccardIndex(num_classes=2)
jaccard(pred, target)

tensor(0.9660)

In [6]:
SMOOTH = 1e-6

In [15]:
def torch_metrics(predict: torch.Tensor, gt: torch.Tensor):
	# predict = predict.squeeze(1)
	intersection = (predict & gt).float().sum((1,2)) # 한쪽이 0이면 0
	print(intersection)
	union = (predict | gt).float().sum((1,2)) # 양쪽다 0 이면 0
	print(union)
	iou = (intersection + SMOOTH) / (union + SMOOTH)
	print(iou)
	threshold = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10 #0.5와 비교
	print(threshold)
	return iou.mean()

In [16]:
torch_metrics(pred, target)

tensor([309., 296., 286., 304., 293., 325., 301., 314., 334., 315.])
tensor([309., 296., 322., 340., 329., 325., 301., 314., 334., 315.])
tensor([1.0000, 1.0000, 0.8882, 0.8941, 0.8906, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])
tensor([1.0000, 1.0000, 0.8000, 0.8000, 0.8000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])


tensor(0.9673)

In [62]:
def torch_dice(predict: torch.Tensor, gt: torch.Tensor):
	# predict = predict.squeeze(1)
	# gt = gt.squeeze(1)
	intersection = (predict & gt).float().sum((1,2)) # 한쪽이 0이면 0
	dice = (2*intersection + SMOOTH) / (predict.sum((1,2)) + gt.sum((1,2)) + SMOOTH)
	# print(dice)
	# threshold = torch.clamp(20 * (dice - 0.5), 0, 10).ceil() / 10 #0.5와 비교
	# print(threshold)
	return dice.mean()

In [63]:
torch_dice(pred, target)

tensor(0.9823)

In [58]:
def dice(pred, gt):
	pred = torch.sigmoid(pred)
	intersection = (pred * target).sum(dim=(1,2))
	union = pred.sum(dim=(1,2)) + target.sum(dim=(1,2))
	dice = 2.0 * (intersection + SMOOTH) / (union + SMOOTH)
	return dice.mean()

In [61]:
dice(pred, target)

tensor(0.6503)

In [60]:
def dice(pred, gt):
	pred = torch.sigmoid(pred)
	intersection = (pred * target).sum(dim=(1,2))
	union = pred.sum(dim=(1,2)) + target.sum(dim=(1,2))
	dice = 2.0 * (torch.sum(intersection) + SMOOTH) / (torch.sum(union) + SMOOTH)
	return dice

In [52]:
def np_iou(predict, target):
	intersection = torch.logical_and(target, predict)
	union = torch.logical_or(target, predict)
	return torch.sum(intersection) / torch.sum(union)

In [80]:
def torch_dice(pred, gt):
	intersection = torch.sum(torch.logical_and(pred, gt))
	union = torch.sum(pred) + torch.sum(gt)
	dice = 2.0 * (intersection + SMOOTH) / (union + SMOOTH)
	return dice

In [82]:
dice(pred, target)

tensor(0.6503)